In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nlp-getting-started/sample_submission.csv
/kaggle/input/nlp-getting-started/train.csv
/kaggle/input/nlp-getting-started/test.csv


In [2]:
import pandas as pd
import numpy as np
tweets = pd.read_csv('../input/nlp-getting-started/train.csv',encoding = 'latin-1')
tweets.head(10)


,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
5,8,NaN,NaN,#RockyFire Update => California Hwy. 20 closed...,1
6,10,NaN,NaN,#flood #disaster Heavy rain causes flash flood...,1
7,13,NaN,NaN,I'm on top of the hill and I can see a fire in...,1
8,14,NaN,NaN,There's an emergency evacuation happening now ...,1
9,15,NaN,NaN,I'm afraid that the tornado is coming to our a...,1


Pre-Processing...

In [3]:
import string
from nltk.corpus import stopwords
from nltk.corpus import words
from nltk.stem import WordNetLemmatizer
from nltk import tokenize

def text_clean(txt):
    Stop = stopwords.words('english')
    no_punc = [char for char in txt if char not in string.punctuation]
    no_punc = ''.join(no_punc)
    return ' '.join([word for word in no_punc.split() if (word.lower() not in Stop)])


In [4]:
%%time
tweets['clean'] = tweets.text.apply(text_clean)
tweets['clean']

CPU times: user 1.22 s, sys: 126 ms, total: 1.35 s
Wall time: 1.35 s


0            Deeds Reason earthquake May ALLAH Forgive us
1                   Forest fire near La Ronge Sask Canada
2       residents asked shelter place notified officer...
3       13000 people receive wildfires evacuation orde...
4       got sent photo Ruby Alaska smoke wildfires pou...
                              ...                        
7608    Two giant cranes holding bridge collapse nearb...
7609    ariaahrary TheTawniest control wild fires Cali...
7610    M194 0104 UTC5km Volcano Hawaii httptcozDtoyd8EbJ
7611    Police investigating ebike collided car Little...
7612    Latest Homes Razed Northern California Wildfir...
Name: clean, Length: 7613, dtype: object

Counter for number and frequency of words:

In [5]:
from collections import Counter
words = tweets[tweets.target==1].clean.apply(lambda x: [word.lower() for word in x.split()])
reg_words = Counter()
for msg in words:
    reg_words.update(msg)
print(reg_words.most_common(50))    

[('fire', 178), ('news', 136), ('via', 121), ('disaster', 117), ('california', 111), ('suicide', 110), ('police', 107), ('amp', 106), ('people', 105), ('killed', 93), ('like', 92), ('hiroshima', 86), ('storm', 85), ('crash', 84), ('fires', 84), ('us', 81), ('families', 81), ('train', 79), ('emergency', 76), ('buildings', 75), ('bomb', 74), ('2', 73), ('two', 71), ('mh370', 71), ('nuclear', 70), ('attack', 69), ('video', 69), ('wildfire', 69), ('get', 66), ('accident', 66), ('bombing', 66), ('one', 65), ('northern', 64), ('burning', 64), ('dead', 63), ('pm', 62), ('legionnaires', 62), ('bomber', 60), ('homes', 58), ('car', 57), ('still', 57), ('war', 57), ('im', 56), ('new', 56), ('atomic', 56), ('obama', 54), ('fatal', 54), ('years', 53), ('collapse', 51), ('may', 50)]


In [6]:
words = tweets[tweets.target==0].clean.apply(lambda x: [word.lower() for word in x.split()])
spam_words = Counter()
for msg in words:
    spam_words.update(msg)
print(spam_words.most_common(50))    

[('like', 253), ('im', 243), ('amp', 192), ('new', 168), ('get', 163), ('dont', 141), ('one', 128), ('body', 112), ('via', 99), ('would', 97), ('video', 96), ('people', 91), ('love', 89), ('2', 86), ('know', 85), ('back', 84), ('time', 83), ('us', 83), ('got', 83), ('see', 82), ('cant', 81), ('emergency', 81), ('full', 81), ('day', 78), ('youtube', 76), ('going', 75), ('still', 72), ('fire', 72), ('go', 67), ('want', 67), ('good', 67), ('think', 66), ('3', 64), ('man', 62), ('world', 62), ('lol', 61), ('rt', 60), ('life', 60), ('u', 59), ('youre', 58), ('first', 58), ('news', 57), ('last', 56), ('burning', 56), ('really', 55), ('way', 55), ('need', 55), ('make', 54), ('work', 54), ('best', 53)]


In [7]:
X = tweets.clean
Y = tweets.target


In [8]:
tweets2 = pd.read_csv('../input/nlp-getting-started/test.csv', encoding = 'latin-1')
tweets2['clean'] = tweets2.text.apply(text_clean)
tweets2['clean']


0                             happened terrible car crash
1       Heard earthquake different cities stay safe ev...
2       forest fire spot pond geese fleeing across str...
3                   Apocalypse lighting Spokane wildfires
4                  Typhoon Soudelor kills 28 China Taiwan
                              ...                        
3258    EARTHQUAKE SAFETY LOS ANGELES ÂÃÃ SAFETY FA...
3259    Storm RI worse last hurricane cityamp3others h...
3260      Green Line derailment Chicago httptcoUtbXLcBIuY
3261    MEG issues Hazardous Weather Outlook HWO httpt...
3262    CityofCalgary activated Municipal Emergency Pl...
Name: clean, Length: 3263, dtype: object

In [9]:
from sklearn.feature_extraction.text import CountVectorizer
v = CountVectorizer()
v.fit(X)
X_train_dtm = v.transform(X)
print(X_train_dtm)


  (0, 1157)	1
  (0, 4540)	1
  (0, 5329)	1
  (0, 6460)	1
  (0, 14680)	1
  (0, 17574)	1
  (0, 21321)	1
  (1, 2982)	1
  (1, 6249)	1
  (1, 6450)	1
  (1, 13746)	1
  (1, 15524)	1
  (1, 18134)	1
  (1, 18424)	1
  (2, 1616)	1
  (2, 5770)	1
  (2, 5866)	1
  (2, 15799)	1
  (2, 15966)	1
  (2, 16164)	1
  (2, 16747)	2
  (2, 17848)	1
  (2, 18824)	2
  (3, 141)	1
  (3, 2935)	1
  :	:
  (7610, 14396)	1
  (7610, 21365)	1
  (7610, 21590)	1
  (7611, 3045)	1
  (7611, 3681)	1
  (7611, 5353)	2
  (7611, 12834)	1
  (7611, 12963)	1
  (7611, 14132)	1
  (7611, 15739)	1
  (7611, 16848)	1
  (7611, 16902)	1
  (7611, 17975)	1
  (7611, 18702)	1
  (7611, 19871)	1
  (7611, 20523)	1
  (7612, 773)	1
  (7612, 2935)	1
  (7612, 7722)	1
  (7612, 12258)	1
  (7612, 13838)	1
  (7612, 15598)	1
  (7612, 15768)	1
  (7612, 17514)	1
  (7612, 21963)	1


In [10]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()
%time nb.fit(X_train_dtm,Y)

CPU times: user 4.91 ms, sys: 0 ns, total: 4.91 ms
Wall time: 5.9 ms


MultinomialNB()

In [11]:
X_test_dtm = v.transform(tweets2.clean)
y_pred_class = nb.predict(X_test_dtm)
print(type(y_pred_class))
sub = pd.DataFrame()
sub['id'] = tweets2.id
sub['target'] = pd.Series(y_pred_class)

sub


<class 'numpy.ndarray'>


,id,target
0,0,1
1,2,1
2,3,1
3,9,1
4,11,1
...,...,...
3258,10861,1
3259,10865,1
3260,10868,1
3261,10874,1


In [12]:
tweets2['prediction'] = sub.target
tweets2
sub.to_csv('./submission.csv', index = False)